In [ ]:
# Import necessary libraries
import pandas as pd
import os
# Set the folder where your 200 CSV files are located
# folder_path = "/content"
folder_path = r"C:\Users\3ddev\OneDrive\Thesis\Characterization\New-SnS2-esserc\SnS2\SnS2\D12" # 🔹 CHANGE THIS TO YOUR CSV FOLDER
# C:\Users\3ddev\OneDrive\Thesis\Characterization\New-SnS2-esserc\SnS2\SnS2\D2
# Initialize an empty dictionary for storing data
data_dict = {}

# Variable to store CH1 Source (only from 1s.csv and 1r.csv)
source_column = None

# Process each pair of files (1S & 1R, 2S & 2R, ..., 100S & 100R)
for i in range(1, 101):
    s_filename = os.path.join(folder_path, f"{i}S.csv")
    r_filename = os.path.join(folder_path, f"{i}R.csv")

    # Check if both files exist
    if not os.path.exists(s_filename) or not os.path.exists(r_filename):
        print(f"Skipping {i}: Missing {s_filename} or {r_filename}")
        continue

    # Read CSV files
    s_df = pd.read_csv(s_filename)
    r_df = pd.read_csv(r_filename)

    # Ensure required columns exist
    required_cols = {'CH1 Current', 'CH1 Source'}
    if not required_cols.issubset(s_df.columns) or not required_cols.issubset(r_df.columns):
        print(f"Skipping {i}: Required columns not found")
        continue

    # Store CH1 Current from each file
    data_dict[f'Current_{i}'] = list(s_df['CH1 Current']) + list(r_df['CH1 Current'])

    # Store CH1 Source **only for 1s and 1r** (append 1r after 1s)
    if i == 1:
        source_column = list(s_df['CH1 Source']) + list(r_df['CH1 Source'])

# Convert dictionary to DataFrame
final_df = pd.DataFrame(dict(CH1_Source=source_column))  # Start with CH1 Source

# Add Current_X columns to the DataFrame
for key, values in data_dict.items():
    max_len = max(len(final_df), len(values))
    final_df = final_df.reindex(range(max_len))  # Extend to match longest column
    final_df[key] = values + [None] * (max_len - len(values))  # Pad with NaN if needed

# Save as CSV
final_filename = os.path.join(folder_path, 'final_combined.csv')
final_df.to_csv(final_filename, index=False)
# files.download("/content/final_combined.csv")
print(f"✅ Processing complete. File saved at: {final_filename}")
